In [81]:
import random
from tabulate import tabulate

# global parameters
p_i = 0.10 # probability of injury
p_p = 0.05 # probability of probing
p_r = 0.5 # probability of retaliation

v = 60   # payoff of winning
c = 100  # cost of serious injury
s = 2    # cost of scratch
u_init = 5  # payoff of saving time and energy (counts down every round)
u_minus = 0.5

num_trials = 100000

class Hawk: 
    def __init__(self, name):
        self.name = name
        self.payoff = 0
        self.currentAction = "C"
        self.injured = False
        self.attacked = False
        
    def move(self):
        if self.injured: 
            self.currentAction = "R"
        else: 
            self.currentAction = "D"
#         print(self.name + ": " + self.currentAction)
    
class Mouse: 
    def __init__(self, name):
        self.name = name
        self.payoff = 0
        self.currentAction = "C"
        self.injured = False
        self.attacked = False
        
    def move(self, u):
        if self.attacked or u == 0:
            self.currentAction = "R"
        else: 
            self.currentAction = "C"
#         print(self.name + ": " + self.currentAction)


class Retaliator: 
    def __init__(self, name):
        self.name = name
        self.payoff = 0
        self.currentAction = "C"
        self.injured = False
        self.attacked = False
        
    def move(self, opponent, u):
        if self.injured or u == 0: 
            self.currentAction = "R"
        elif opponent.currentAction == "D":
            if random.random() <= p_r:
                self.currentAction = "D"
            else: 
                self.currentAction = "C"
        elif opponent.currentAction == "C":
            self.currentAction = "C"
#         print("      " + self.name + ": " + self.currentAction)

        
class ProberRetaliator:
    def __init__(self, name):
        self.name = name
        self.payoff = 0
        self.currentAction = "C"
        self.injured = False
        self.attacked = False
        self.probe = False
        
    def move(self, opponent, u):
        if self.injured or u == 0: 
            self.currentAction = "R"

        elif not self.probe: # if it's not currently probing
            if opponent.currentAction == "D":
                if random.random() <= p_r:
                    self.currentAction = "D"      # retaliates

            elif opponent.currentAction == "C":
                if random.random() <= p_p:
                    self.currentAction = "D"      # probes
                    self.probe = True
                else:
                    self.currentAction = "C"

        else: # if it is currently probing
            if opponent.currentAction == "D": # opponent has retaliated
                self.currentAction = "C"  # reverts back to C
                self.probe = False
            elif opponent.currentAction == "C": 
                self.currentAction = "D"  # takes advantage and keeps probing

#         print(self.name + ": " + self.currentAction)


def check(contestant, opponent): 
    if contestant.currentAction == "D":
        if not isinstance(opponent, Mouse):
            if random.random() <= p_i: # 1 in 10 chance
                opponent.payoff -= c # serious injury
                opponent.injured = True
            else: 
                opponent.payoff -= s # scratch
        else: 
            opponent.attacked = True

    if contestant.currentAction == "R":
        opponent.payoff += v
        return False
    
    return True

        
def main():

    """
    Hawk vs. Hawk
    """
    total = 0
    total2= 0
    
    for _ in range(num_trials): 
        p1 = Hawk("p1")
        p2 = Hawk("p2")
        
        u = u_init  #saved-time payoff starts at u_init
        
        while True: 
            # p1 goes first
            p1.move()
            
            # check payoffs
            if not check(p1, p2): 
                break
            
            # p2 goes next
            p2.move()
            
#             u -= u_minus
            
            # check payoffs
            if not check(p2, p1): 
                break

            # decrease time-saved payoff
            u -= u_minus    

        # uninjured players get time-saved payoff
        if u < 0: 
            u = 0
        if not p1.injured:
            p1.payoff += u
        if not p2.injured: 
            p2.payoff += u 
            
        # add final payoffs to totals
        total += p1.payoff
        total2 += p2.payoff
    
    # calculate average of all trials, average of p1 and p2
#     print("Hawk vs. Hawk avg payoff: " + str((total+total2)/(2*num_trials)))
#     print("\n")
    hh = (total+total2)/(2*num_trials)


    
    """
    Hawk vs. Mouse
    """
    total = 0
    total2 = 0
    for _ in range(num_trials): 
        p1 = Hawk("p1")
        p2 = Mouse("p2")
        
        u = u_init
        
        while True: 
            # p1 moves
            p1.move()
            
            # check payoffs
            if not check(p1, p2):
                break
                
#             u -= u_minus
            
            # p2 moves
            p2.move(u)
                
            # check payoffs
            if not check(p2, p1): 
                break
            
            # decrease time-saved payoff
            u -= u_minus
                
        # uninjured players get time-saved payoff
        if u < 0: 
            u = 0
        if not p1.injured:
            p1.payoff += u
        if not p2.injured: 
            p2.payoff += u 
            
        # add final payoffs to totals
        total += p1.payoff
        total2 += p2.payoff
    
    # calculate average of num_trials trials
#     print("Hawk vs. Mouse avg payoff: " + str(total/num_trials))
#     print("   Mouse's payoff: " + str(total2/num_trials))
#     print("\n")
    hm1 = total/num_trials
    mh2 = total2/num_trials
    
    
    """
    Mouse vs. Hawk
    """
    total = 0
    total2 = 0
    
    for _ in range(num_trials): 
        p1 = Mouse("p1")
        p2 = Hawk("p2")
        
        u = u_init
        
        while True: 
            # p1 moves
            p1.move(u)
            
            # check payoffs
            if not check(p1, p2): 
                break
                
#             u -= u_minus
            
            # p2 moves
            p2.move()
                
            # check payoffs
            if not check(p2, p1):
                break
            
            # decrease time-saved payoff
            u -= u_minus
                
        # uninjured players get time-saved payoff
        if u < 0: 
            u = 0
        if not p1.injured:
            p1.payoff += u 
        if not p2.injured:
            p2.payoff += u
            
        # add final payoffs to totals
        total += p1.payoff
        total2 += p2.payoff
    
    # calculate average of num_trials trials
#     print("Mouse vs. Hawk avg payoff: " + str(total/num_trials))
#     print("   Hawk's payoff: " + str(total2/num_trials))
#     print("\n")
    mh1 = total/num_trials
    hm2 = total2/num_trials
    
    
    
    """
    Mouse vs. Mouse
    """
    total = 0
    total2 = 0
    
    for _ in range(num_trials): 
        p1 = Mouse("p1")
        p2 = Mouse("p2")
        
        u = u_init
        
        while True:             
            # p1 moves
            p1.move(u)
            
            # check payoffs
            if not check(p1, p2): 
                break
                
#             u -=u_minus
            
            # p2 moves
            p2.move(u)
                
            # check payoffs
            if not check(p2, p1):
                break
            
            # decrease time-saved payoff
            u -= u_minus
                
        # uninjured players get time-saved payoff
        if u < 0: 
            u = 0
        if not p1.injured: 
            p1.payoff += u 
        if not p2.injured:
            p2.payoff += u
            
        # add final payoffs to totals
        total += p1.payoff
        total2 += p2.payoff
    
    # calculate average of num_trials trials
#     print("Mouse vs. Mouse avgavg payoff: " + str((total+total2)/(2*num_trials)))
#     print("\n")
    mm = (total+total2)/(2*num_trials)
    
    
    
    """
    Retaliator vs. Mouse
    """
    total = 0
    total2 = 0
    
    for _ in range(num_trials): 
        p1 = Retaliator("p1")
        p2 = Mouse("p2")
        
        u = u_init
        
        while True: 
            # p1 moves
            p1.move(p2, u)

            # check payoffs
            if not check(p1, p2):
                break
            
#             u -= u_minus
            
            # p2 moves
            p2.move(u)

            # check payoffs
            if not check(p2, p1):
                break
            
            # decrease time-saved payoff
            u -= u_minus
                
        # uninjured players get time-saved payoff
        if u < 0: 
            u = 0
        if not p1.injured: 
            p1.payoff += u 
        if not p2.injured:
            p2.payoff += u
            
        # add final payoffs to totals
        total += p1.payoff
        total2 += p2.payoff
    
    # calculate average of num_trials trials
#     print("Retaliator vs. Mouse avg payoff: " + str(total/num_trials))
#     print("   Mouse's payoff: "+ str(total2/num_trials))
#     print("\n")
    rm1 = total/num_trials
    mr2 = total2/num_trials
    
        
        
    """
    Mouse vs. Retaliator
    """
    total = 0
    total2 = 0
    
    for _ in range(num_trials): 
        p1 = Mouse("p1")
        p2 = Retaliator("p2")
        
        u = u_init
        
        while True: 
            # p1 moves
            p1.move(u)

            # check payoffs
            if not check(p1, p2):
                break
                
#             u -= u_minus
            
            # p2 moves
            p2.move(p1, u)

            # check payoffs
            if not check(p2, p1):
                break
            
            # decrease time-saved payoff
            u -= u_minus
                
        # uninjured players get time-saved payoff
        if u < 0: 
            u = 0
        if not p1.injured: 
            p1.payoff += u 
        if not p2.injured:
            p2.payoff += u
            
        # add final payoffs to totals
        total += p1.payoff
        total2 += p2.payoff
    
    # calculate average of num_trials trials
#     print("Mouse vs. Retaliator avg payoff: " + str(total/num_trials))
#     print("   Retaliator's payoff: " + str(total2/num_trials))
#     print("\n")
    mr1 = total/num_trials
    rm2 = total2/num_trials
        
    
        
    """
    Retaliator vs. Hawk
    """
    total = 0
    total2 = 0 
    
    for _ in range(num_trials): 
        p1 = Retaliator("p1")
        p2 = Hawk("p2")
        
        u = u_init
        
        while True: 
            # p1 moves
            p1.move(p2, u)
            
            # check payoffs
            if not check(p1, p2):
                break
            
#             u -= u_minus
            
            # p2 moves
            p2.move()
                
            # check payoffs
            if not check(p2, p1): 
                break
                            
            # decrease time-saved payoff
            u -= u_minus
                
        # uninjured players get time-saved payoff
        if u < 0: 
            u = 0
        if not p1.injured: 
            p1.payoff += u 
        if not p2.injured: 
            p2.payoff += u
            
        # add final payoffs to totals
        total += p1.payoff
        total2 += p2.payoff
    
    # calculate average of num_trials trials
#     print("Retaliator vs. Hawk avg payoff: " + str(total/num_trials))
#     print("   Hawk's payoff: " + str(total2/num_trials))
#     print("\n")
    rh1 = total/num_trials
    hr2 = total2/num_trials
    
    
    
    """
    Hawk vs. Retaliator
    """
    total = 0
    total2 = 0
    
    for _ in range(num_trials): 
        p1 = Hawk("p1")
        p2 = Retaliator("p2")
        
        u = u_init
        
        while True: 
            # p1 moves
            p1.move()
            
            # check payoffs
            if not check(p1, p2):
                break
            
#             u -= u_minus
            
            # p2 moves
            p2.move(p1, u)
                
            # check payoffs
            if not check(p2, p1): 
                break
                            
            # decrease time-saved payoff
            u -= u_minus
                
        # uninjured players get time-saved payoff
        if u < 0: 
            u = 0
        if not p1.injured: 
            p1.payoff += u 
        if not p2.injured: 
            p2.payoff += u
            
        # add final payoffs to totals
        total += p1.payoff
        total2 += p2.payoff
    
    # calculate average of num_trials trials
#     print("Hawk vs. Retaliator avg payoff: " + str(total/num_trials))
#     print("   Retaliator's payoff: " + str(total2/num_trials))
#     print("\n")
    hr1 = total/num_trials
    rh2 = total2/num_trials
    
    
    
    """
    Retaliator vs. Retaliator
    """
    total = 0
    total2 = 0
    
    for _ in range(num_trials): 
        p1 = Retaliator("p1")
        p2 = Retaliator("p2")
        
        u = u_init
        
        while True: 
            # p1 moves
            p1.move(p2, u)
            
            # check payoffs
            if not check(p1, p2):
                break
            
#             u -= u_minus
            
            # p2 moves
            p2.move(p1, u)
                
            # check payoffs
            if not check(p2, p1): 
                break
                            
            # decrease time-saved payoff
            u -= u_minus
                
        # uninjured players get time-saved payoff
        if u < 0: 
            u = 0
        if not p1.injured: 
            p1.payoff += u 
        if not p2.injured: 
            p2.payoff += u
            
        # add final payoffs to totals
        total += p1.payoff
        total2 += p2.payoff
    
    # calculate average of num_trials trials
#     print("Retaliator vs. Retaliator avg payoff: " + str((total+total2)/(2*num_trials)))
#     print("\n")
    rr = (total+total2)/(2*num_trials)
    
    
    
    """
    Prober-Retaliator vs. Mouse 
    """
    total = 0
    total2 = 0
    
    for _ in range(num_trials): 
        p1 = ProberRetaliator("p1")
        p2 = Mouse("p2")
        
        u = u_init
        
        while True: 
            # p1 moves
            p1.move(p2, u) 
            
            # check payoffs
            if not check(p1, p2):
                break
            
#             u -= u_minus
            
            # p2 moves
            p2.move(u)
                
            # check payoffs
            if not check(p2, p1):
                break
            
            # decrease time-saved payoff
            u -= u_minus
                
        # uninjured players get time-saved payoff
        if u < 0: 
            u = 0
        if not p1.injured: 
            p1.payoff += u 
        if not p2.injured:
            p2.payoff += u
            
        # add final payoffs to totals
        total += p1.payoff
        total2 += p2.payoff
    
    # calculate average of num_trials trials
#     print("Prober-Retaliator vs. Mouse avg payoff: " + str(total/num_trials))
#     print("   Mouse's payoff: " + str(total2/num_trials))
#     print("\n")
    pm1 = total/num_trials
    mp2 = total2/num_trials
    
    
    
    """
    Mouse vs. Prober-Retaliator
    """
    total = 0
    total2 = 0 
    
    for _ in range(num_trials): 
        p1 = Mouse("p1")
        p2 = ProberRetaliator("p2")
        
        u = u_init
        
        while True: 
            # p1 moves
            p1.move(u)
            
            # check payoffs
            if not check(p1, p2):
                break
            
#             u -= u_minus
            
            # p2 moves
            p2.move(p1, u)
                
            # check payoffs
            if not check(p2, p1): 
                break
                            
            # decrease time-saved payoff
            u -= u_minus
                
        # uninjured players get time-saved payoff
        if u < 0: 
            u = 0
        if not p1.injured: 
            p1.payoff += u 
        if not p2.injured: 
            p2.payoff += u
            
        # add final payoffs to totals
        total += p1.payoff
        total2 += p2.payoff
    
    # calculate average of num_trials trials
#     print("Mouse vs. Prober-Retaliator avg payoff: " + str(total/num_trials))
#     print("   Prober-Retaliator's payoff: " + str(total2/num_trials))
#     print("\n")
    mp1 = total/num_trials
    pm2 = total2/num_trials
    
    
    
    """
    Prober-Retaliator vs. Hawk
    """
    total = 0
    total2 = 0
    
    for _ in range(num_trials): 
        p1 = ProberRetaliator("p1")
        p2 = Hawk("p2")
        
        u = u_init
        
        while True: 
            # p1 moves
            p1.move(p2, u)
            
            # check payoffs
            if not check(p1, p2):
                break
            
#             u -= u_minus
            
            # p2 moves
            p2.move()
                
            # check payoffs
            if not check(p2, p1): 
                break
                            
            # decrease time-saved payoff
            u -= u_minus
                
        # uninjured players get time-saved payoff
        if u < 0: 
            u = 0
        if not p1.injured: 
            p1.payoff += u 
        if not p2.injured: 
            p2.payoff += u
            
        # add final payoffs to totals
        total += p1.payoff
        total2 += p2.payoff
    
    # calculate average of num_trials trials
#     print("Prober-Retaliator vs. Hawk avg payoff: " + str(total/num_trials))
#     print("   Hawk's payoff: " + str(total2/num_trials))
#     print("\n")
    ph1 = total/num_trials
    hp2 = total2/num_trials
    
    
    
    """
    Hawk vs. Prober-Retaliator
    """
    total = 0
    total2 = 0
    
    for _ in range(num_trials): 
        p1 = Hawk("p1")
        p2 = ProberRetaliator("p2")
        
        u = u_init
        
        while True: 
            # p1 moves
            p1.move()
            
            # check payoffs
            if not check(p1, p2):
                break
            
#             u -= u_minus
            
            # p2 moves
            p2.move(p1, u)
                
            # check payoffs
            if not check(p2, p1): 
                break
                            
            # decrease time-saved payoff
            u -= u_minus
                
        # uninjured players get time-saved payoff
        if u < 0: 
            u = 0
        if not p1.injured: 
            p1.payoff += u 
        if not p2.injured: 
            p2.payoff += u
            
        # add final payoffs to totals
        total += p1.payoff
        total2 += p2.payoff
    
    # calculate average of num_trials trials
#     print("Hawk vs. Prober-Retaliator avg payoff: " + str(total/num_trials))
#     print("   Prober-Retaliator's payoff: " + str(total2/num_trials))
#     print("\n")
    hp1 = total/num_trials
    ph2 = total2/num_trials
    
    
    
    """
    Prober-Retaliator vs. Retaliator 
    """
    total = 0
    total2 = 0
    
    for _ in range(num_trials): 
        p1 = ProberRetaliator("p1")
        p2 = Retaliator("p2")
        
        u = u_init
        
        while True: 
            p1.move(p2, u)

            # check payoffs
            if not check(p1, p2):
                break
            
#             u -= u_minus
            
            # p2 moves
            p2.move(p1, u)
                
            # check payoffs
            if not check(p2, p1):
                break
            
            # decrease time-saved payoff
            u -= u_minus
                
        # uninjured players get time-saved payoff
        if u < 0: 
            u = 0
        if not p1.injured: 
            p1.payoff += u 
        if not p2.injured: 
            p2.payoff += u
            
        # add final payoffs to totals
        
        total += p1.payoff
        total2 += p2.payoff
    
    # calculate average of num_trials trials
#     print("Prober-Retaliator vs. Retaliator avg payoff: " + str(total/num_trials))
#     print("   Retaliator's payoff: " + str(total2/num_trials))
#     print("\n")
    pr1 = total/num_trials
    rp2 = total2/num_trials
    
    
    
    """
    Retaliator vs. Prober-Retaliator
    """
    total = 0
    total2 = 0 
    
    for _ in range(num_trials): 
        p1 = Retaliator("p1")
        p2 = ProberRetaliator("p2")
        
        u = u_init
        
        while True: 
            # p1 moves
            p1.move(p2, u)
            
            # check payoffs
            if not check(p1, p2):
                break
            
#             u -= u_minus
            
            # p2 moves
            p2.move(p1, u)
                
            # check payoffs
            if not check(p2, p1): 
                break
                            
            # decrease time-saved payoff
            u -= u_minus
                
        # uninjured players get time-saved payoff
        if u < 0: 
            u = 0
        if not p1.injured: 
            p1.payoff += u 
        if not p2.injured: 
            p2.payoff += u
            
        # add final payoffs to totals
        total += p1.payoff
        total2 += p2.payoff
    
    # calculate average of num_trials trials
#     print("Retaliator vs. Prober-Retaliator avg payoff: " + str(total/num_trials))
#     print("   Prober-Retaliator's payoff: " + str(total2/num_trials))
#     print("\n")
    rp1 = total/num_trials
    pr2 = total2/num_trials
    
    
    
    """
    Prober-Retaliator vs. Prober-Retaliator 
    """
    total = 0
    total2 = 0
    
    for _ in range(num_trials): 
        p1 = ProberRetaliator("p1")
        p2 = ProberRetaliator("p2")
        
        u = u_init
        
        while True: 
            # p1 moves
            p1.move(p2, u)
            
            # check payoffs
            if not check(p1, p2):
                break
                                    
            # p2 moves
            p2.move(p1, u)

            # check payoffs
            if not check(p2, p1):
                break
                    
            # decrease time-saved payoff
            u -= u_minus
                
        # uninjured players get time-saved payoff
        if u < 0: 
            u = 0
        if not p1.injured: 
            p1.payoff += u 
        if not p2.injured: 
            p2.payoff += u
            
        # add final payoffs to totals
        total += p1.payoff
        total2 += p2.payoff
    
    # calculate average of num_trials trials
#     print("PR vs. PR avg payoff: " + str((total+total2)/(2*num_trials)))
#     print("\n")
    pp = (total+total2)/(2*num_trials)
    
    
    
    """
    Print out-puts as table
    """
    table = [["","Mouse","Hawk","Retaliator","PR"], 
             ["Mouse",mm,(mh1+mh2)/2,(mr1+mr2)/2,(mp1+mp2)/2],
             ["Hawk",(hm1+hm2)/2,hh,(hr1+hr2)/2,(hp1+hp2)/2],
             ["Retaliator",(rm1+rm2)/2,(rh1+rh2)/2,rr,(rp1+rp2)/2],
             ["PR",(pm1+pm2)/2,(ph1+ph2)/2,(pr1+pr2)/2,pp]]
    print(tabulate(table, headers = "firstrow"))
    

if __name__ == "__main__":
    main()

              Mouse      Hawk    Retaliator        PR
----------  -------  --------  ------------  --------
Mouse       30         4.75         30       19.0548
Hawk        64.75    -27.5648       18.8159  -2.13518
Retaliator  30       -50.4443       30       19.8295
PR          43.1232  -37.0577       27.7057  18.7722
